# Flight price prediction

## Questions
a) Does price vary with Airlines?

b) How is the price affected when tickets are bought in just 1 or 2 days before departure?

c) Does ticket price change based on the departure time and arrival time?

d) How the price changes with change in Source and Destination?

e) How does the ticket price vary between Economy and Business class?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [42]:
df_clean = pd.read_csv('./data/Clean_Dataset.csv')
df_business = pd.read_csv('./data/business.csv')
df_business['stop'] = df_business['stop'].str.replace(r'\s+', ' ', regex=True)
df_economy = pd.read_csv('./data/economy.csv')

In [43]:
df_clean

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...
300148,300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [44]:
df_business

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop,23:55,Mumbai,"46,690"
...,...,...,...,...,...,...,...,...,...,...,...
93482,31-03-2022,Vistara,UK,822,09:45,Chennai,10h 05m,1-stop,19:50,Hyderabad,"69,265"
93483,31-03-2022,Vistara,UK,826,12:30,Chennai,10h 25m,1-stop,22:55,Hyderabad,"77,105"
93484,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop,20:55,Hyderabad,"79,099"
93485,31-03-2022,Vistara,UK,828,07:00,Chennai,10h 00m,1-stop,17:00,Hyderabad,"81,585"


In [45]:
df_economy

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"
...,...,...,...,...,...,...,...,...,...,...,...
206769,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,697"
206770,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,709"
206771,31-03-2022,Vistara,UK,826,12:30,Chennai,20h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:05,Hyderabad,"8,640"
206772,31-03-2022,Vistara,UK,822,09:45,Chennai,23h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:05,Hyderabad,"8,640"


## Removing useless data

## Missing and duplicates values

In [46]:
# Checking if there are duplicates in table df_clean
df_clean.duplicated().any()

False

In [47]:
# Checking if there are duplicates in table df_business
df_business.duplicated().any()

False

In [48]:
# Checking if there are duplicates in table df_economy
df_economy.duplicated().any()

True

In [49]:
# Removing duplicates in df_economy (2)
df_economy.drop_duplicates()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"
...,...,...,...,...,...,...,...,...,...,...,...
206769,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,697"
206770,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,709"
206771,31-03-2022,Vistara,UK,826,12:30,Chennai,20h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:05,Hyderabad,"8,640"
206772,31-03-2022,Vistara,UK,822,09:45,Chennai,23h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,09:05,Hyderabad,"8,640"


## Standardise and normalize data

## Encoding categorical variables

In [50]:
df_clean_encoded = pd.get_dummies(df_clean, columns=['airline', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class'], dtype=int)
df_clean_encoded

,Unnamed: 0,flight,duration,days_left,price,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,...,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy
0,0,SG-8709,2.17,1,5953,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
1,1,SG-8157,2.33,1,5953,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,1
2,2,I5-764,2.17,1,5956,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,3,UK-995,2.25,1,5955,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,4,UK-963,2.33,1,5955,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300148,300148,UK-822,10.08,49,69265,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
300149,300149,UK-826,10.42,49,77105,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
300150,300150,UK-832,13.83,49,79099,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
300151,300151,UK-828,10.00,49,81585,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [51]:
df_business_encoded = pd.get_dummies(df_business, columns=['airline', 'ch_code', 'from', 'stop', 'to'], dtype=int)
df_business_encoded

,date,num_code,dep_time,time_taken,arr_time,price,airline_Air India,airline_Vistara,ch_code_AI,ch_code_UK,...,stop_1-stop Via Surat,stop_1-stop Via VTZ,stop_2+-stop,stop_non-stop,to_Bangalore,to_Chennai,to_Delhi,to_Hyderabad,to_Kolkata,to_Mumbai
0,11-02-2022,868,18:00,02h 00m,20:00,"25,612",1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
1,11-02-2022,624,19:00,02h 15m,21:15,"25,612",1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
2,11-02-2022,531,20:00,24h 45m,20:45,"42,220",1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,11-02-2022,839,21:25,26h 30m,23:55,"44,450",1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,11-02-2022,544,17:15,06h 40m,23:55,"46,690",1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93482,31-03-2022,822,09:45,10h 05m,19:50,"69,265",0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
93483,31-03-2022,826,12:30,10h 25m,22:55,"77,105",0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
93484,31-03-2022,832,07:05,13h 50m,20:55,"79,099",0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
93485,31-03-2022,828,07:00,10h 00m,17:00,"81,585",0,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [52]:
df_economy_encoded = pd.get_dummies(df_economy, columns=['airline', 'ch_code', 'from', 'stop', 'to'], dtype=int)
df_economy_encoded

,date,num_code,dep_time,time_taken,arr_time,price,airline_Air India,airline_AirAsia,airline_GO FIRST,airline_Indigo,...,stop_1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia VTZ\n\t\t\t\t\t\t\t\t\t\t\t\t,stop_1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Vishakhapatnam\n\t\t\t\t\t\t\t\t\t\t\t\t,stop_2+-stop,stop_non-stop,to_Bangalore,to_Chennai,to_Delhi,to_Hyderabad,to_Kolkata,to_Mumbai
0,11-02-2022,8709,18:55,02h 10m,21:05,"5,953",0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,11-02-2022,8157,06:20,02h 20m,08:40,"5,953",0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,11-02-2022,764,04:25,02h 10m,06:35,"5,956",0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,11-02-2022,995,10:20,02h 15m,12:35,"5,955",0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,11-02-2022,963,08:50,02h 20m,11:10,"5,955",0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206769,31-03-2022,832,07:05,13h 50m,20:55,"7,697",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
206770,31-03-2022,832,07:05,13h 50m,20:55,"7,709",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
206771,31-03-2022,826,12:30,20h 35m,09:05,"8,640",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
206772,31-03-2022,822,09:45,23h 20m,09:05,"8,640",0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Outlier data

### Removing

### Transforming

## Data visualization

### Name of feature1 [chart/hist/dot etc...]

### Name of feature2 [chart/hist/dot etc...]

.

.

.

### Name of featureN [chart/hist/dot etc...]

## Varble effect on the outcome

### Categorical

### Numerical

## Developing new features

## Selecting most important features [not necessary]